In [1]:
import igraph

In [3]:
import pandas as pd

In [19]:
def parse_gdf(file_name):
    with open(file_name) as f:
        rows = [line.strip().split(',') for line in f.readlines()]
        #print(rows)
        metadata = [row for row in rows if len(row) > 3]
        graph_data = [row for row in rows if len(row) == 3]
        return pd.DataFrame(metadata[1:], columns=metadata[0]), pd.DataFrame(graph_data[1:], columns=graph_data[0])
            
df_metadata, df_graph = parse_gdf('channelnet_seeds1_nodes13_2019_05_01-17_26_00.gdf')
df_metadata

,nodedef>name VARCHAR,label VARCHAR,isSeed VARCHAR,seedRank INT,subscriberCount INT,videoCount INT,viewCount(100s) INT,country VARCHAR,publishedAt VARCHAR,daysactive INT
0,UCuaFvcY4MhZY3U43mMt1dYQ,Walt Disney Studios,yes,1,2499412,947,10893293,US,2008-11-18T20:34:44.000Z,3816
1,UCAwm8rSWCoi94powYWnhz6Q,Disney Family,no,,228620,336,206188,US,2005-11-24T06:58:39.000Z,4906
2,UCB0ABGbdSggBXLF0UdN4d5A,The Muppets,no,,702469,277,3494887,not set,2009-11-20T01:57:11.000Z,3450
3,UC_976xMxPgzIa290Hqtk-9g,Walt Disney Animation Studios,no,,3511643,347,22436777,US,2008-03-25T16:32:01.000Z,4054
4,UC_IRYSp4auq7hKLvziWVH6w,Pixar,no,,2552333,521,12507275,US,2006-09-19T17:42:01.000Z,4607
5,UC4IEmIzWf_X4lEPtGm7kgzQ,Disney Movies,no,,509143,310,898677,not set,2007-03-31T02:38:25.000Z,4415
6,UC_5niPa-d35gg88HaS7RrIw,Disney,no,,3632930,776,22566493,not set,2012-02-11T12:07:58.000Z,2636
7,UCbFnGyIcAhDrj-90OzvI7kw,Oh My Disney,no,,344389,450,963857,US,2013-07-11T06:09:13.000Z,2120
8,UCZSVJrC2Hnu92n2Lhez3KgA,Disney Style,no,,547115,477,1534234,not set,2012-02-27T20:05:43.000Z,2620
9,UC5K8SEF_7GQBedXIjtXLCRg,Mickey Mouse,no,,3008416,249,20604685,not set,2008-03-12T17:52:01.000Z,4067


In [20]:
df_graph

,edgedef>node1 VARCHAR,node2 VARCHAR,directed BOOLEAN
0,UCuaFvcY4MhZY3U43mMt1dYQ,UCAwm8rSWCoi94powYWnhz6Q,true
1,UCuaFvcY4MhZY3U43mMt1dYQ,UCB0ABGbdSggBXLF0UdN4d5A,true
2,UCuaFvcY4MhZY3U43mMt1dYQ,UC_976xMxPgzIa290Hqtk-9g,true
3,UCuaFvcY4MhZY3U43mMt1dYQ,UC_IRYSp4auq7hKLvziWVH6w,true
4,UCuaFvcY4MhZY3U43mMt1dYQ,UC4IEmIzWf_X4lEPtGm7kgzQ,true
5,UCAwm8rSWCoi94powYWnhz6Q,UC_5niPa-d35gg88HaS7RrIw,true
6,UCAwm8rSWCoi94powYWnhz6Q,UCbFnGyIcAhDrj-90OzvI7kw,true
7,UCAwm8rSWCoi94powYWnhz6Q,UCZSVJrC2Hnu92n2Lhez3KgA,true
8,UCAwm8rSWCoi94powYWnhz6Q,UC5K8SEF_7GQBedXIjtXLCRg,true
9,UCAwm8rSWCoi94powYWnhz6Q,UCB0ABGbdSggBXLF0UdN4d5A,true


In [21]:
#help(igraph)